In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow.keras import regularizers,  layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Conv1D, Conv2D, BatchNormalization, Activation, Add,
                                     MaxPooling2D, GlobalAveragePooling2D, Dense, Reshape, MaxPooling1D, Flatten, Dropout, LSTM, TimeDistributed, Bidirectional,SeparableConv2D)

# Importar datos para  entrenamiento

In [ ]:
# Rutas de los datos procesados
TRAIN_PATH = "../data_train/processed_audio/train.npz"
TEST_PATH = "../data_train/processed_audio/test.npz"

# Cargar datos
data_train = np.load(TRAIN_PATH)
data_test = np.load(TEST_PATH)

X_train, y_train = data_train["X"], data_train["y"]
X_test, y_test = data_test["X"], data_test["y"]

# Verificar forma de los datos
print("Datos cargados correctamente:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# Imprimir distribución de etiquetas en el conjunto de entrenamiento
labels_train = np.argmax(y_train, axis=1)
unique_labels_train, counts_train = np.unique(labels_train, return_counts=True)
print("\nDistribución de etiquetas en y_train:")
for label, count in zip(unique_labels_train, counts_train):
    print(f"Etiqueta {label}: {count} ejemplos")

# Imprimir distribución de etiquetas en el conjunto de prueba
labels_test = np.argmax(y_test, axis=1)
unique_labels_test, counts_test = np.unique(labels_test, return_counts=True)
print("\nDistribución de etiquetas en y_test:")
for label, count in zip(unique_labels_test, counts_test):
    print(f"Etiqueta {label}: {count} ejemplos")

# Crear `tf.data.Dataset` para optimizar el entrenamiento
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE  # Ajuste automático del prefetching

train_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(len(X_train))  # Mezclar datos
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

print("\nDatasets creados y listos para entrenar.")


## Modelo: NN simple


In [ ]:
# Parámetros
num_features = X_train.shape[1]  # Longitud de entrada (1 segundo de audio a 16kHz)
num_classes = y_train.shape[1]   # 2 categorías (cry, noise)


# Definir el modelo
model = Sequential([
    Input(shape=(num_features,)),  # Capa de entrada
    Dense(32, activation='relu'), # Capa oculta
    Dense(16, activation='relu'), # Capa oculta
    Dense(num_classes, activation='softmax')  # Capa de salida
])

# Compilar el modelo
model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Mostrar resumen del modelo
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True) # Detener si no hay mejora en la validación tras 3 épocas. Se  queda con los mejores pesos.
mc = ModelCheckpoint("../models/not_compressed_audio/model_NN.h5", save_best_only=True, monitor='val_loss') # Guardar el mejor modelo basado en la validación.

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

## Modelo: NN avanzada

In [ ]:
num_features = X_train.shape[1]
num_classes = y_train.shape[1]

model = Sequential([
    Input(shape=(num_features,)),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(8, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
])

model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_audio/model_NNA.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

## Modelo: CNN simple

In [ ]:
num_features = X_train.shape[1]
num_classes = y_train.shape[1]

model = Sequential([
    Reshape((num_features, 1), input_shape=(num_features,), name="add_channel"),

    Conv1D(16, kernel_size=9, strides=4, activation='relu', name="conv1"),
    MaxPooling1D(pool_size=4, name="pool1"),

    Conv1D(8, kernel_size=9, strides=4, activation='relu', name="conv2"),
    MaxPooling1D(pool_size=4, name="pool2"),

    Flatten(name="flatten"),
    Dense(16, activation='relu', name="dense1"),

    Dense(2, activation='softmax', name="output")
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_audio/model_CNN1.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

## Modelo: CNN avanzada

In [ ]:
num_features = X_train.shape[1]
num_classes  = y_train.shape[1]


model = Sequential([
    Reshape((num_features, 1), input_shape=(num_features,), name="add_channel"),

    Conv1D(64, 9, activation='relu', padding='same'),
    MaxPooling1D(pool_size=4, name="pool1"),

    Conv1D(32, 9, activation='relu', padding='same'),
    MaxPooling1D(pool_size=4, name="pool2"),

    Conv1D(16, 9, activation='relu', padding='same'),
    MaxPooling1D(pool_size=4, name="pool3"),

    Flatten(name="flatten"),
    Dense(64, activation='relu', name="dense1"),

    Dense(num_classes, activation='softmax', name="output")
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_audio/model_CNNA.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

## Modelos para MFCC

In [ ]:
# Rutas de los datos procesados
TRAIN_PATH = "../data_train/processed_MFCC/train.npz"
TEST_PATH = "../data_train/processed_MFCC/test.npz"

# Cargar datos procesados
data_train = np.load(TRAIN_PATH)
data_test = np.load(TEST_PATH)

X_train, y_train = data_train["X"], data_train["y"]
X_test, y_test = data_test["X"], data_test["y"]

# Verificar forma de los datos
print("Datos cargados correctamente:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# Imprimir distribución de etiquetas en el conjunto de entrenamiento
labels_train = np.argmax(y_train, axis=1)
unique_labels_train, counts_train = np.unique(labels_train, return_counts=True)
print("\nDistribución de etiquetas en y_train:")
for label, count in zip(unique_labels_train, counts_train):
    print(f"Etiqueta {label}: {count} ejemplos")

# Imprimir distribución de etiquetas en el conjunto de prueba
labels_test = np.argmax(y_test, axis=1)
unique_labels_test, counts_test = np.unique(labels_test, return_counts=True)
print("\nDistribución de etiquetas en y_test:")
for label, count in zip(unique_labels_test, counts_test):
    print(f"Etiqueta {label}: {count} ejemplos")


BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (
    tf.data.Dataset.from_tensor_slices((tf.cast(X_train, tf.float32), y_train))
    .cache()
    .shuffle(buffer_size=len(X_train))
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((tf.cast(X_test, tf.float32), y_test))
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

print("Datasets listos para entrenar.")


In [ ]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

model = Sequential([

        Input(shape=input_shape),
        Reshape(input_shape + (1,)),


        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        Flatten(),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()



In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_MFCC/model_CNN.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)

In [ ]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

model = Sequential([
        Input(shape=input_shape),
        Reshape(input_shape + (1,)),

        Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),


        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        Flatten(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
])


model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()



In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint("../models/not_compressed_MFCC/model_CNNA.h5", save_best_only=True, monitor='val_loss')

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[es, mc]
)